# Huggingface inference code

In [1]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch
import soundfile as sf

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

inputs = processor(text="Hello, my dog is cute.", return_tensors="pt")

# load xvector containing speaker's voice characteristics from a dataset
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

sf.write("speech.wav", speech.numpy(), samplerate=16000)

# 모델 local에 저장하는 코드

In [3]:
model.save_pretrained("microsoft/speecht5_tts")
processor.save_pretrained("microsoft/speecht5_tts")
vocoder.save_pretrained("microsoft/speecht5_hifigan")

# handler.py code

In [9]:
import soundfile as sf
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch

class CustomTTSHandler:
    def __init__(self, model_dir, vocoder_dir):
        self.processor = SpeechT5Processor.from_pretrained(model_dir)
        self.model = SpeechT5ForTextToSpeech.from_pretrained(model_dir)
        self.vocoder = SpeechT5HifiGan.from_pretrained(vocoder_dir)

        # Load speaker embeddings
        embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
        self.speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

    def predict(self, text):
        inputs = self.processor(text=text, return_tensors="pt")
        with torch.no_grad():
            speech = self.model.generate_speech(inputs["input_ids"], self.speaker_embeddings, vocoder=self.vocoder)
        return speech

# 모델 디렉토리 지정
model_dir = "/Users/lhe339/Documents/GitHub/TTS/SpeechT5/microsoft/speecht5_tts"
vocoder_dir = "/Users/lhe339/Documents/GitHub/TTS/SpeechT5/microsoft/speecht5_hifigan"

# CustomTTSHandler 인스턴스 생성
tts_handler = CustomTTSHandler(model_dir, vocoder_dir)

# 텍스트를 음성으로 변환
text = "hi, my name is honey."
speech_output = tts_handler.predict(text)

# 결과를 WAV 파일로 저장
sf.write('output.wav', speech_output.numpy(), samplerate=16000)